In [16]:
import os
import pandas as pd
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

directory: str = '/data/HSC_generated/HSC_generated_v1/ddpm/g_band_ddpm'
window: str = '30px_diameter'
data = []

for filename in os.listdir(directory):
    if filename.endswith(".cat"):
        file_path = os.path.join(directory, filename)
        with open(file_path, "r") as f:
            
            object_id = filename.split("_")[2].split(".")[0]
            
            segmented_filepath = f'{directory}/test_segmented_{object_id}.fits'
            img = fits.getdata(segmented_filepath)
            img_size = img.shape
            
            center_i = 32
            center_j = 32
            radius = int(int(window[:2])/2)
            in_center = []
            
            for i in range(img_size[0]):
                for j in range(img_size[1]):
                    dist = np.sqrt((i - center_i)**2 + (j - center_j)**2)
                    if dist <= radius:
                        if img[i, j] not in in_center:
                            if img[i, j] != 0:
                                in_center.append(img[i, j])
            num_in_center = len(in_center)
            
            param_names = []
            
            for line in f:
                if line.startswith('#'):
                    param_line = line.strip().split()
                    param_names.append(param_line[2])
                if not line.startswith('#'):
                    val_line = line.strip().split()
                    row_data = []
                    for i in val_line:
                        row_data.append(i)
                    row_data.insert(0, num_in_center)
                    row_data.insert(0, object_id)
                    data.append(row_data)
            param_names.insert(0, 'NUMBER_IN_CENTER')
            param_names.insert(0, 'object_id')

In [17]:
df = pd.DataFrame(data, columns=param_names)

In [18]:
if not os.path.exists(f'/data/HSC_generated/HSC_generated_v1/ddpm/g_band_ddpm/{window}_test_set_subset/'):
    os.makedirs(f'/data/HSC_generated/HSC_generated_v1/ddpm/g_band_ddpm/{window}_test_set_subset/')
df.to_csv(f'/data/HSC_generated/HSC_generated_v1/ddpm/g_band_ddpm/{window}_test_set_subset/segmented_image_data_v1.csv', index=False)

In [19]:
param_names

['object_id',
 'NUMBER_IN_CENTER',
 'NUMBER',
 'PETRO_RADIUS',
 'X_IMAGE',
 'Y_IMAGE',
 'XMIN_IMAGE',
 'XMAX_IMAGE',
 'YMIN_IMAGE',
 'YMAX_IMAGE',
 'ISOAREA_IMAGE',
 'ISOAREA_WORLD',
 'A_IMAGE',
 'B_IMAGE',
 'ERRCXX_IMAGE',
 'ERRCYY_IMAGE',
 'ERRCXY_IMAGE',
 'THETA_IMAGE',
 'THETA_WORLD',
 'MU_MAX',
 'ELLIPTICITY',
 'FLUX_RADIUS',
 'SPHEROID_SERSICN']

In [20]:
df

,object_id,NUMBER_IN_CENTER,NUMBER,PETRO_RADIUS,X_IMAGE,Y_IMAGE,XMIN_IMAGE,XMAX_IMAGE,YMIN_IMAGE,YMAX_IMAGE,...,B_IMAGE,ERRCXX_IMAGE,ERRCYY_IMAGE,ERRCXY_IMAGE,THETA_IMAGE,THETA_WORLD,MU_MAX,ELLIPTICITY,FLUX_RADIUS,SPHEROID_SERSICN
0,1859,2,1,4.62,32.8612,32.0729,25,40,24,40,...,2.387,12974.69,11954.34,28.7833,-80.12,-80.12,-6.4327,0.041,2.581,1.113
1,1859,2,2,8.58,12.6084,8.7956,7,18,3,14,...,2.204,722.0573,835.3883,243.7778,-33.75,-33.75,-3.8111,0.143,3.726,1.045
2,1859,2,3,7.26,37.9666,59.5385,28,49,42,64,...,2.470,2304.339,1308.959,2610.121,-54.21,-54.21,-5.3721,0.597,4.739,2.075
3,38,1,1,10.56,31.2987,36.0049,27,35,30,42,...,1.951,251.1728,128.8498,35.04464,-82.45,-82.45,-2.8459,0.285,9.854,2.377
4,1204,1,1,6.60,24.5169,32.0755,18,31,27,38,...,2.212,1723.48,2184.564,321.2614,-20.60,-20.60,-4.5821,0.082,2.944,1.415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4195,884,2,2,8.58,38.3473,60.7004,33,43,56,64,...,2.010,253.0172,336.7902,49.4593,-14.08,-14.08,-2.7903,0.135,4.051,0.599
4196,884,2,3,10.56,11.2520,50.1258,9,13,49,52,...,0.870,168.4141,299.9836,16.19391,-2.91,-2.91,-2.3124,0.250,2.563,1.476
4197,884,2,4,10.56,50.8002,35.7895,45,55,32,39,...,1.732,226.2088,357.0865,-0.5314739,0.42,0.42,-2.7164,0.193,4.393,1.443
4198,14,1,1,6.60,31.3199,33.3791,17,45,26,41,...,2.905,329.2089,1140.993,409.3913,-15.55,-15.55,-3.6268,0.515,6.453,0.854


In [21]:
import os
import pandas as pd
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt

In [32]:
directory: str = '/data/HSC_generated/HSC_generated_v1/cvae/g_band_cvae'
window: str = '30px_diameter'
data = []

for filename in os.listdir(directory):
    if filename.endswith(".cat"):
        file_path = os.path.join(directory, filename)
        with open(file_path, "r") as f:
            
            object_id = filename.split("_")[2].split(".")[0]
            
            segmented_filepath = f'{directory}/test_segmented_{object_id}.fits'
            img = fits.getdata(segmented_filepath)
            img_size = img.shape
            
            center_i = 32
            center_j = 32
            radius = int(int(window[:2])/2)
            in_center = []
            
            for i in range(img_size[0]):
                for j in range(img_size[1]):
                    dist = np.sqrt((i - center_i)**2 + (j - center_j)**2)
                    if dist <= radius:
                        if img[i, j] not in in_center:
                            if img[i, j] != 0:
                                in_center.append(img[i, j])
            num_in_center = len(in_center)
            
            param_names = []
            
            for line in f:
                if line.startswith('#'):
                    param_line = line.strip().split()
                    param_names.append(param_line[2])
                if not line.startswith('#'):
                    val_line = line.strip().split()
                    row_data = []
                    for i in val_line:
                        row_data.append(i)
                    row_data.insert(0, num_in_center)
                    row_data.insert(0, object_id)
                    data.append(row_data)
            param_names.insert(0, 'NUMBER_IN_CENTER')
            param_names.insert(0, 'object_id')

In [33]:
df = pd.DataFrame(data, columns=param_names)

In [34]:
if not os.path.exists(f'/data/HSC_generated/HSC_generated_v1/cvae/g_band_cvae/{window}_test_set_subset/'):
    os.makedirs(f'/data/HSC_generated/HSC_generated_v1/cvae/g_band_cvae/{window}_test_set_subset/')
df.to_csv(f'/data/HSC_generated/HSC_generated_v1/cvae/g_band_cvae/{window}_test_set_subset/segmented_image_data_v1.csv', index=False)

In [35]:
param_names

['object_id',
 'NUMBER_IN_CENTER',
 'NUMBER',
 'PETRO_RADIUS',
 'X_IMAGE',
 'Y_IMAGE',
 'XMIN_IMAGE',
 'XMAX_IMAGE',
 'YMIN_IMAGE',
 'YMAX_IMAGE',
 'ISOAREA_IMAGE',
 'ISOAREA_WORLD',
 'A_IMAGE',
 'B_IMAGE',
 'ERRCXX_IMAGE',
 'ERRCYY_IMAGE',
 'ERRCXY_IMAGE',
 'THETA_IMAGE',
 'THETA_WORLD',
 'MU_MAX',
 'ELLIPTICITY',
 'FLUX_RADIUS',
 'SPHEROID_SERSICN']

In [36]:
df

,object_id,NUMBER_IN_CENTER,NUMBER,PETRO_RADIUS,X_IMAGE,Y_IMAGE,XMIN_IMAGE,XMAX_IMAGE,YMIN_IMAGE,YMAX_IMAGE,...,B_IMAGE,ERRCXX_IMAGE,ERRCYY_IMAGE,ERRCXY_IMAGE,THETA_IMAGE,THETA_WORLD,MU_MAX,ELLIPTICITY,FLUX_RADIUS,SPHEROID_SERSICN
0,1859,1,1,4.62,32.4209,32.5445,20,44,22,44,...,2.302,5376934,7486096,2119392,-2.52,-2.52,-6.3638,0.080,2.396,1.065
1,1859,1,2,4.62,11.7758,59.1453,8,16,51,64,...,1.726,11013.38,4256.037,887.6227,-87.69,-87.69,-0.9962,0.310,2.519,0.300
2,38,3,1,4.62,10.2198,11.4309,1,39,1,43,...,5.679,8879.871,6663.387,220.1571,89.66,89.66,-3.6245,0.130,6.132,0.936
3,38,3,2,5.94,32.3393,31.5803,16,55,21,55,...,6.351,1869.535,2712.145,-113.6472,2.78,2.78,-3.6601,0.136,8.134,2.907
4,38,3,3,5.94,26.2502,19.7295,17,36,13,24,...,2.058,3304.55,9295.282,5294.955,-12.77,-12.77,-2.7298,0.458,3.661,1.349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11142,14,2,4,6.60,56.0368,59.8257,54,58,58,62,...,0.923,166.3405,217.1611,156.5645,-37.48,-37.48,1.1972,0.410,2.200,3.930
11143,14,2,5,4.62,62.4289,59.6957,60,64,56,64,...,1.151,54100.74,17496.59,-1735.192,80.20,80.20,-2.0461,0.404,1.940,0.304
11144,14,2,6,5.28,28.1370,56.1932,25,31,53,59,...,1.524,685.2977,666.7624,-57.46102,75.72,75.72,0.3911,0.072,2.143,0.303
11145,1834,2,1,5.94,32.6418,33.2370,15,49,14,55,...,5.943,1185.224,915.6861,565.2405,-60.28,-60.28,-4.7647,0.224,9.323,0.918


In [15]:
import os
import pandas as pd
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from tqdm import tqdm

directory: str = '/data/HSC_generated/HSC_generated_v1/g_band'
window: str = '30px_diameter'
data = []

for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".cat"):
        file_path = os.path.join(directory, filename)
        with open(file_path, "r") as f:
            
            object_id = filename.split("_")[2].split(".")[0]
            
            segmented_filepath = f'{directory}/test_segmented_{object_id}.fits'
            img = fits.getdata(segmented_filepath)
            img_size = img.shape
            
            center_i = 32
            center_j = 32
            radius = int(int(window[:2])/2)
            in_center = []
            
            for i in range(img_size[0]):
                for j in range(img_size[1]):
                    dist = np.sqrt((i - center_i)**2 + (j - center_j)**2)
                    if dist <= radius:
                        if img[i, j] not in in_center:
                            if img[i, j] != 0:
                                in_center.append(img[i, j])
            num_in_center = len(in_center)
            
            param_names = []
            
            for line in f:
                if line.startswith('#'):
                    param_line = line.strip().split()
                    param_names.append(param_line[2])
                if not line.startswith('#'):
                    val_line = line.strip().split()
                    row_data = []
                    for i in val_line:
                        row_data.append(i)
                    row_data.insert(0, num_in_center)
                    row_data.insert(0, object_id)
                    data.append(row_data)
            param_names.insert(0, 'NUMBER_IN_CENTER')
            param_names.insert(0, 'object_id')

100%|██████████| 372351/372351 [17:10<00:00, 361.41it/s]


In [16]:
df = pd.DataFrame(data, columns=param_names)

In [17]:
if not os.path.exists(f'/data/HSC_generated/HSC_generated_v1/cvae/g_band_cvae/{window}_test_set_subset/'):
    os.makedirs(f'/data/HSC_generated/HSC_generated_v1/cvae/g_band_cvae/{window}_test_set_subset/')
df.to_csv(f'/data/HSC_generated/HSC_generated_v1/cvae/g_band_cvae/{window}_test_set_subset/segmented_image_data_v1.csv', index=False)

In [18]:
param_names

['object_id',
 'NUMBER_IN_CENTER',
 'NUMBER',
 'PETRO_RADIUS',
 'X_IMAGE',
 'Y_IMAGE',
 'XMIN_IMAGE',
 'XMAX_IMAGE',
 'YMIN_IMAGE',
 'YMAX_IMAGE',
 'ISOAREA_IMAGE',
 'ISOAREA_WORLD',
 'A_IMAGE',
 'B_IMAGE',
 'ERRCXX_IMAGE',
 'ERRCYY_IMAGE',
 'ERRCXY_IMAGE',
 'THETA_IMAGE',
 'THETA_WORLD',
 'MU_MAX',
 'ELLIPTICITY',
 'FLUX_RADIUS',
 'SPHEROID_SERSICN']

In [7]:
# from astropy.io import fits
# import matplotlib.pyplot as plt

# for filename in os.listdir(directory)[:50]:
#     if filename.startswith("test_segmented"):
#         file_path = os.path.join(directory, filename)
#         print(filename)
#         img = fits.getdata(file_path)
#         plt.figure()
#         plt.imshow(img)
#         plt.show()

In [8]:
# obj_ids = np.unique(df['object_id'])
# for object_id in obj_ids[:30]:
#     segmented_filepath = f'{directory}/test_segmented_{object_id}_step1.fits'
#     original_filepath = f'{directory}/{object_id}_step1.fits'
#     segmented_img = fits.getdata(segmented_filepath)
#     original_img = fits.getdata(original_filepath)
#     fig, (segmented_ax, original_ax) = plt.subplots(1, 2)
#     segmented_ax.imshow(segmented_img)
#     original_ax.imshow(original_img)

In [9]:
# must clear prevous segmented and cat files if you want to make a new batch

In [10]:
# detailed galaxies might be seen as multiple galaxies, as we need to consider the separation between galaxies
# it might be hard to keep track of

# run num_in_center script on segmented files and match magnitudes by object id to the df

# plot magnitude vs petro rad 
# also make histograms when the df is full